In [13]:
import pandas as pd
from  helpers_db import get_engine, run_sql
from backtesting import Backtest, Strategy
from sqlalchemy.sql import text

In [53]:
engine = get_engine()
with engine.begin() as conn:
  data = run_sql(conn, """WITH ma AS (
	SELECT symbol, dt,
		AVG(close) OVER(PARTITION BY symbol ORDER BY symbol, dt ROWS 16 PRECEDING) AS MA_slow,
		AVG(close) OVER(PARTITION BY symbol ORDER BY symbol, dt ROWS 64 PRECEDING) AS MA_high,
		CASE 
			WHEN AVG(close) OVER(PARTITION BY symbol ORDER BY symbol, dt ROWS 16 PRECEDING) >
				AVG(close) OVER(PARTITION BY symbol ORDER BY symbol, dt ROWS 64 PRECEDING) THEN 1
			WHEN AVG(close) OVER(PARTITION BY symbol ORDER BY symbol, dt ROWS 16 PRECEDING) <
				AVG(close) OVER(PARTITION BY symbol ORDER BY symbol, dt ROWS 64 PRECEDING) THEN -1
		END MA_dir,
		open, close, low, high, volume
	FROM ticks_4h
	WHERE symbol = 'ADAUSD' AND dt::DATE >= '2024-03-01'
	ORDER BY dt DESC
)
, ma_reversal AS (
	SELECT symbol, dt, open, close, low, high, volume, MA_dir,
		LAG(MA_dir) OVER(PARTITION BY symbol ORDER BY symbol, dt DESC) AS MA_dir_prev,
		CASE
			WHEN MA_dir <> LAG(MA_dir) OVER(PARTITION BY symbol ORDER BY symbol, dt DESC) AND
				MA_dir = -1 
			THEN 1
			WHEN MA_dir <> LAG(MA_dir) OVER(PARTITION BY symbol ORDER BY symbol, dt DESC) AND
				MA_dir = 1 
			THEN -1
		END AS signal
	FROM ma
	ORDER by dt ASC
)
SELECT *
FROM ma_reversal""")
  df = pd.DataFrame(data)
  df.rename(columns={"close": "Close", "open": "Open", "low": "Low", "high": "High", "volume": "Volume", "dt": "datetime"}, inplace=True)
  df.set_index("datetime", inplace = True)
print(df)

                              Close      High       Low  ma_dir  ma_dir_prev  \
datetime                                                                       
2024-03-01 00:00:00+00:00  0.673000  0.683476  0.654322     NaN          NaN   
2024-03-01 04:00:00+00:00  0.664100  0.675478  0.664100     NaN          NaN   
2024-03-01 08:00:00+00:00  0.686015  0.689947  0.663000     NaN          NaN   
2024-03-01 12:00:00+00:00  0.679231  0.695000  0.676002     NaN          NaN   
2024-03-01 16:00:00+00:00  0.691305  0.695173  0.674143     NaN          NaN   
...                             ...       ...       ...     ...          ...   
2024-04-13 17:00:00+00:00  0.451668  0.503959  0.439416    -1.0         -1.0   
2024-04-13 21:00:00+00:00  0.445144  0.454869  0.400001    -1.0         -1.0   
2024-04-14 01:00:00+00:00  0.446300  0.448764  0.433500    -1.0         -1.0   
2024-04-14 09:00:00+00:00  0.456652  0.472072  0.446660    -1.0         -1.0   
2024-04-14 13:00:00+00:00  0.466216  0.4

In [54]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    # def init(self):
    #     pass

    # def next(self):
    #     current_signal = self.data.signal[-1]

    #     if current_signal == 1:
    #         if not self.position:
    #             self.buy()
    #     elif current_signal == -1:
    #         if self.position:
    #             self.position.close()

    n1 = 16
    n2 = 64

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

bt = Backtest(df, SmaCross, cash=10000, commission=.0025, exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-03-01 00:00...
End                       2024-04-14 13:00...
Duration                     44 days 13:00:00
Exposure Time [%]                   64.794007
Equity Final [$]                 12286.782205
Equity Peak [$]                  12501.650581
Return [%]                          22.867822
Buy & Hold Return [%]              -30.725706
Return (Ann.) [%]                  431.426068
Volatility (Ann.) [%]              459.006181
Sharpe Ratio                         0.939913
Sortino Ratio                       11.973379
Calmar Ratio                        21.359013
Max. Drawdown [%]                  -20.198783
Avg. Drawdown [%]                   -5.743159
Max. Drawdown Duration       23 days 21:00:00
Avg. Drawdown Duration        4 days 17:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                       21.11142
Worst Trade [%]                     -8.612388
Avg. Trade [%]                    